In [2]:
search_for = 135
start_from = 0
threads = 10
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.018292903900146484
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 175
all cases: 723027151
type: [1, 1, 1, 1, 135] 135
cases of this type: 332150625
100000 43.06775021782499
200000 69.01181750774437
300000 68.5601098912983
400000 68.09739723862539
500000 71.07092256220534
600000 85.59150403016463
700000 77.14136607284868
800000 73.00362376568732
900000 74.8214470892364
1000000 74.21675677751925
1100000 71.38055332797359
1200000 64.91948152681377
1300000 26.16991315589292
1400000 72.10279749057175
1500000 70.77447659587568
1600000 71.01684494505365
1700000 72.04138008196996
1800000 71.19571038649843
1900000 76.4798225335715
2000000 69.24466975933828
2100000 70.35815976855052
2200000 72.63256948231465
2300000 73.43207445604767
2400000 71.74485300109106
2500000 22.50776386483104
2600000 75.68911509501419
2700000 97.75364007070752
2800000 95.35051317675271
2900000 94.82707137898937
3000000 98.17975237773862
310

30100000 76.34950214086652
30200000 73.54850844288383
30300000 74.21812127478807
30400000 75.1557181668837
30500000 71.36281508451555
30600000 74.05918823782966
30700000 71.19578827008772
30800000 15.854486901742693
30900000 73.43680724162473
31000000 68.77500462635997
31100000 71.04780412165624
31200000 76.63333088187278
31300000 74.31571100923196
31400000 73.15643499029973
31500000 78.18672665517579
31600000 72.78261527387372
31700000 77.81066859757313
31800000 26.484724539177382
31900000 77.16287761133349
32000000 75.93790129588909
32100000 77.44663214950609
32200000 71.40739244621265
32300000 33.42312198416323
32400000 79.31547516806239
32500000 77.17479820335726
32600000 76.83005311740808
32700000 77.43205500450271
32800000 78.44945928423266
32900000 76.65427770283262
33000000 72.98062404893565
33100000 68.92054447586277
33200000 56.34261684539816
33300000 38.06632120373317
33400000 72.20198630546678
33500000 76.36803654152116
33600000 73.76705454921208
33700000 75.45327666962503


60500000 65.70912847222846
60600000 67.55612311986125
60700000 64.34626876534494
60800000 69.6824617087805
60900000 67.84717062565521
61000000 69.12294010939266
61100000 30.036640627494506
61200000 73.06458041503524
61300000 77.35151469757966
61400000 72.15935930934968
61500000 73.48684956430724
61600000 74.90710228572982
61700000 72.84681898941176
61800000 72.03315351503241
61900000 77.9914508240681
62000000 35.04697647249885
62100000 63.455773023364515
62200000 69.38128300288672
62300000 70.21396380756639
62400000 69.56840627175063
62500000 66.0397092786088
62600000 65.77492654171098
62700000 63.20436005740257
62800000 18.80766711030544
62900000 66.16914817701989
63000000 65.33761843853657
63100000 63.73839315132348
63200000 68.54085798187606
63300000 71.33073486874802
63400000 68.16113292196799
63500000 53.22020666016965
63600000 49.005885706295146
63700000 73.8393585727733
63800000 68.57733447372085
63900000 72.47223988817964
64000000 72.91617241340212
64100000 72.40750990050417
64

90900000 65.56662443917968
91000000 67.61323488120144
91100000 66.97951636336494
91200000 66.65929219786318
91300000 71.41695641600379
91400000 68.67515374329929
91500000 69.82062795519663
91600000 71.1509025273629
91700000 51.80031641246843
91800000 35.30521004159735
91900000 60.82515701662325
92000000 64.22911538535678
92100000 61.8928942453757
92200000 62.2050752212264
92300000 19.941230636868212
92400000 60.00127318070431
92500000 63.333361692733256
92600000 62.48100824565609
92700000 60.842175590780776
92800000 45.86197380666706
92900000 42.22951521231814
93000000 63.43293958171059
93100000 67.24593749227985
93200000 67.91435591295506
93300000 67.66034786379127
93400000 65.8445437615272
93500000 72.73476600037826
93600000 71.12132558281036
93700000 63.935791138351256
93800000 69.8870427118472
93900000 66.31415077039665
94000000 64.00796780864766
94100000 70.59854275581337
94200000 40.212466957730086
94300000 45.36734870337814
94400000 55.5304310201594
94500000 58.99861989651528
94

120400000 72.94367129705104
120500000 68.30344968698019
120600000 70.26620772709457
120700000 71.89809520933169
120800000 67.11244753131184
120900000 65.21206071191179
121000000 65.87326235154939
121100000 63.80265647954909
121200000 61.198036282919006
121300000 64.58862211609022
121400000 62.0445503022209
121500000 21.948894702165866
121600000 60.622678799850306
121700000 61.72692817295442
121800000 34.11387970760363
121900000 44.231785022409206
122000000 62.34761672524155
122100000 48.40660409235875
122200000 37.59384537867067
122300000 61.25479236656616
122400000 63.24521439036863
122500000 58.876914359101875
122600000 61.17480034477734
122700000 66.89386279601844
122800000 68.72704277302664
122900000 65.71680188122177
123000000 66.09239945093586
123100000 64.1646663549827
123200000 70.1452300425272
123300000 66.61151024890782
123400000 66.01335301050841
123500000 61.69865776771368
123600000 62.96338721727916
123700000 57.150185303182184
123800000 58.04206263944869
123900000 59.3704

...

639500000 2.2257986235904945
639600000 1.9524934216300727
best so far: 0
type: [1, 1, 3, 5, 9] 135
cases of this type: 7381125
639700000 2.0620217444192073
639800000 1.7618688131664682
639900000 1.798877665034246
640000000 1.7456061506905258
640100000 1.8145905735519692
640200000 1.889364564108587
640300000 1.892927819280543
640400000 1.8132562509525836
640500000 1.7509860905620687
640600000 1.772207495048674
640700000 1.7446678495988786
640800000 1.7164707262694887
640900000 1.8440610896422804
641000000 1.9920067548994262
641100000 1.8803191255639438
641200000 1.7457975323359192
641300000 1.6603891064516092
641400000 1.667156642531472
641500000 1.6080042302067759
641600000 1.68636762715824
641700000 1.730063411057582
641800000 1.8243278443341673
641900000 1.9175768444047359
642000000 1.8022689753923955
642100000 1.6431083520393452
642200000 1.5874744192446286
642300000 1.5253151565054317
642400000 1.5580396010413589
642500000 1.6101532342060938
642600000 1.710983273889929
642700000 1.

667000000 2.6338858234249596
667100000 2.3917391762253515
667200000 2.704010607456936
667300000 2.5748841254577144
667400000 2.728962092498835
667500000 2.730820304779131
667600000 2.624763721149633
667700000 2.8283096185457866
best so far: 0
type: [1, 1, 5, 3, 9] 135
cases of this type: 4428675
667800000 1.7902187463615993
667900000 1.0935596659824134
668000000 1.0037467725149074
668100000 1.1249387351882512
668200000 1.046072690680343
668300000 1.1213526903442748
668400000 1.0967785459270167
668500000 1.0339513022688152
668600000 1.0583404979679605
668700000 1.1124144108086156
668800000 1.0918466275234402
668900000 1.0060817797753547
669000000 0.9729643546149684
669100000 0.9957214354191561
669200000 1.072063780890596
669300000 1.208921016992696
669400000 1.1744224343366423
669500000 1.0086234750635081
669600000 1.0414184634322914
669700000 1.0931935097797334
669800000 1.4240145814901186
669900000 1.3799776872702338
670000000 1.2318024388626911
670100000 1.1137232171565064
670200000 

691500000 0.7913059555098136
691600000 1.612773220073402
691700000 1.1233074446089262
691800000 1.126375759585054
691900000 1.8402592476709563
692000000 0.9380123056608214
692100000 1.1506483005311072
692200000 1.4970338293633165
692300000 1.179980625901891
692400000 1.5535893669015652
692500000 1.1544098693728824
692600000 0.9992487247637153
692700000 1.8409643145464334
692800000 0.9624230956974765
692900000 0.9038413420669748
693000000 1.065782635206575
693100000 0.9775264176542761
693200000 1.1935183782891856
693300000 0.8404150334359983
693400000 0.8974008992812952
693500000 1.2592254057222882
693600000 1.247802015368088
693700000 0.8183596375764688
693800000 1.4326196980211148
693900000 1.1255419927898824
694000000 1.0679174803518832
694100000 1.0471952096002355
694200000 1.0157063086874996
694300000 1.0640090729757368
694400000 1.2615033737224022
694500000 1.1500021009610475
694600000 1.2340249369920808
694700000 1.0062943704885436
694800000 1.304144492698441
694900000 0.84072494

best so far: 0
type: [1, 9, 1, 3, 5] 135
cases of this type: 151875
713000000 0.1617629647593048
713100000 0.10216369228760665
best so far: 0
type: [1, 9, 1, 5, 3] 135
cases of this type: 91125
713200000 0.07784993224551479
best so far: 0
type: [1, 9, 1, 15, 1] 135
cases of this type: 30375
best so far: 0
type: [1, 9, 3, 1, 5] 135
cases of this type: 50625
best so far: 0
type: [1, 9, 3, 5, 1] 135
cases of this type: 10125
713300000 0.05913376253307329
best so far: 0
type: [1, 9, 5, 1, 3] 135
cases of this type: 18225
best so far: 0
type: [1, 9, 5, 3, 1] 135
cases of this type: 6075
best so far: 0
type: [1, 9, 15, 1, 1] 135
cases of this type: 2025
best so far: 0
type: [1, 15, 1, 1, 9] 135
cases of this type: 98415
713400000 0.08449818504113818
best so far: 0
type: [1, 15, 1, 3, 3] 135
cases of this type: 32805
best so far: 0
type: [1, 15, 1, 9, 1] 135
cases of this type: 10935
best so far: 0
type: [1, 15, 3, 1, 3] 135
cases of this type: 10935
best so far: 0
type: [1, 15, 3, 3, 1] 135


best so far: 0
type: [5, 9, 3, 1, 1] 135
cases of this type: 81
best so far: 0
type: [5, 27, 1, 1, 1] 135
cases of this type: 27
best so far: 0
type: [9, 1, 1, 1, 15] 135
cases of this type: 50625
best so far: 0
type: [9, 1, 1, 3, 5] 135
cases of this type: 16875
best so far: 0
type: [9, 1, 1, 5, 3] 135
cases of this type: 10125
best so far: 0
type: [9, 1, 1, 15, 1] 135
cases of this type: 3375
723000000 0.0003352084615555075
best so far: 0
type: [9, 1, 3, 1, 5] 135
cases of this type: 5625
best so far: 0
type: [9, 1, 3, 5, 1] 135
cases of this type: 1125
best so far: 0
type: [9, 1, 5, 1, 3] 135
cases of this type: 2025
best so far: 0
type: [9, 1, 5, 3, 1] 135
cases of this type: 675
best so far: 0
type: [9, 1, 15, 1, 1] 135
cases of this type: 225
best so far: 0
type: [9, 3, 1, 1, 5] 135
cases of this type: 1875
best so far: 0
type: [9, 3, 1, 5, 1] 135
cases of this type: 375
best so far: 0
type: [9, 3, 5, 1, 1] 135
cases of this type: 75
best so far: 0
type: [9, 5, 1, 1, 3] 135
cases